# We're going to compare the two binning methods for all of our galaxies.

Let's import all the necessary modules and then make a list of usable FITS file paths 

In [1]:
from new_Galaxy import SpiralGalaxy as SG1
from newer_Galaxy import SpiralGalaxy as SG2

INFO: No release version set. Setting default to MPL-7


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
def append_directory(path):
    fits_list = []
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".fits.gz"):
                element = str(path) + str(file)
                fits_list.append(element)
    return fits_list

In [4]:
fits_list = append_directory('/raid5/homes/sshamsi/sas/mangawork/manga/sandbox/galaxyzoo3d/v1_0_0/')

In [5]:
usable_gals = np.load('usable_gals.npy')

In [6]:
usable_paths = []

for path in fits_list:
    if path.split('/')[-1].split('_')[0] in usable_gals:
        usable_paths.append(path)

Let us now compare both the binning methods.

In [7]:
gal_list1 = []
gal_list2 = []

for path in usable_paths:
    gal1 = SG1(path)
    gal2 = SG2(path)
    
    gal_list1.append(gal1)
    gal_list2.append(gal2)

/homes/sshamsi/.local/lib/python3.5/site-packages/marvin/tools/maps.py:272: MarvinUserWarning: mismatch between file version=DR15 and object release=MPL-7. Setting object release to DR15
  marvin.core.exceptions.MarvinUserWarning)
/homes/sshamsi/.local/lib/python3.5/site-packages/marvin/utils/general/general.py:354: MarvinUserWarning: more than one plate-ifu found for mangaid=1-211017. Using the one with the highest SN2.
  MarvinUserWarning)


In [8]:
for i in range(len(usable_paths)):
    gal_list1[i].form_global_df()
    gal_list2[i].form_global_df()

/raid5/homes/sshamsi/galaxy_zoo/GZ3D_spiral_analysis/new_Galaxy.py:134: RuntimeWarning: invalid value encountered in double_scalars
  x['SFR'] = (self.delta * (ha_flux**3.36) * (hb_flux**-2.36)) / self.spax_area
/raid5/homes/sshamsi/galaxy_zoo/GZ3D_spiral_analysis/new_Galaxy.py:134: RuntimeWarning: invalid value encountered in double_scalars
  x['SFR'] = (self.delta * (ha_flux**3.36) * (hb_flux**-2.36)) / self.spax_area


In [9]:
for gal in gal_list1:
    map_ha = gal.maps.getMap('emline_gflux', channel='ha_6564')
    map_hb = gal.maps.getMap('emline_gflux', channel='hb_4862')

    ha_snr = map_ha.snr.flatten()
    hb_snr = map_hb.snr.flatten()

    gal.df['S/N $H_{\\alpha}$'] = ha_snr
    gal.df['S/N $H_{\\beta}$'] = hb_snr

In [ ]:
for gal in gal_list2:
    map_ha = gal.maps.getMap('emline_gflux', channel='ha_6564')
    map_hb = gal.maps.getMap('emline_gflux', channel='hb_4862')

    ha_snr = map_ha.snr.flatten()
    hb_snr = map_hb.snr.flatten()

    gal.df['S/N $H_{\\alpha}$'] = ha_snr
    gal.df['S/N $H_{\\beta}$'] = hb_snr

In [ ]:
tot_sfrs1 = []
tot_sfr_stdvs1 = []

tot_sfrs2 = []
tot_sfr_stdvs2 = []

for i in range(len(usable_paths)):
    df1 = gal_list1[i].df
    df2 = gal_list2[i].df
    
    df1 = df1.dropna()
    df2 = df2.dropna()
    
    df1 = df1[(df1['$r/r_e$'] <= 1) & (df1['$r/r_e$'] >= 0.1)]
    df1 = df1[(df1.Comp == 0) & (df1.AGN == 0) & (df1.Seyfert == 0) & (df1.Liner == 0)]
    df1 = df1[(df1['S/N $H_{\\alpha}$'] >= 3) & (df1['S/N $H_{\\beta}$'] >= 3)]
    
    df2 = df2[(df2['$r/r_e$'] <= 1) & (df2['$r/r_e$'] >= 0.1)]
    df2 = df2[(df2.Comp == 0) & (df2.AGN == 0) & (df2.Seyfert == 0) & (df2.Liner == 0)]
    df2 = df2[(df2['S/N $H_{\\alpha}$'] >= 3) & (df2['S/N $H_{\\beta}$'] >= 3)]
    
    sfr1 = df1.SFR.sum()
    sfr_stdv1 = np.sqrt(gal_list1[i].get_var(df1.index, mode='sfr', avg=False))
    
    sfr2, sfr_stdv2 = gal_list2[i].get_sfr(df2.index, avg=False)
    
    tot_sfrs1.append(sfr1)
    tot_sfr_stdvs1.append(sfr_stdv1)
    
    tot_sfrs2.append(sfr2)
    tot_sfr_stdvs2.append(sfr_stdv2)